TRABALHO BD

A partir de webscrapping, ir buscar o número de ids escolhido (máx 20) de termos à escolha do utilizador e importar as informações para a base de dados

WEBSCRAPPING

In [234]:
query = input("Defina o que pretende pesquisar [bacteria/yeast/...]: ")
def url_get(item):
    url_list = []
    url = "https://www.ncbi.nlm.nih.gov/gene/?term={}".format(query)
    url_list.append(url)
    return url_list
    
url_get(0)

['https://www.ncbi.nlm.nih.gov/gene/?term=yeast']

In [235]:
import requests, sys, json
content = []
for url in url_get(0):
    r = requests.get(url)
    content.append(r.content)
# print(content)

In [236]:
import requests
from bs4 import BeautifulSoup

# Parsing the HTML
for c in content:
    soup = BeautifulSoup(r.content, 'html.parser')
    res = soup.get_text()
# print(res)

In [237]:
import re
existe = re.findall(r'\s*ID:\s*\d*(?=\D)', res, re.DOTALL)
id_genes = ','.join(existe)
id_g = re.sub(r'\s*ID:\n*\s+', '', id_genes)
id_gene = id_g.split(',')
id_gene


['424122',
 '29789',
 '26245415',
 '20918',
 '24563533',
 '39874648',
 '4839978',
 '42049400',
 '42048491',
 '36516274',
 '35606568',
 '35598500',
 '24564880',
 '24564814',
 '13446414',
 '5477914',
 '65082716',
 '109860985',
 '8047674',
 '4292']

In [238]:
number_of_genes = int(input("Número de genes a obter (máx: 20): "))
print(number_of_genes)
genes = id_gene[0:number_of_genes]
print(genes)

10
['424122', '29789', '26245415', '20918', '24563533', '39874648', '4839978', '42049400', '42048491', '36516274']


In [7]:
#BUSCAR LINKS DOS GENES

In [239]:
import time
def url_get_id(item):
    url_id = []
    for id in genes:
        url = f"https://www.ncbi.nlm.nih.gov/nuccore/{id}"
        url_id.append(url)
    return url_id
url_get_id(genes)

['https://www.ncbi.nlm.nih.gov/nuccore/424122',
 'https://www.ncbi.nlm.nih.gov/nuccore/29789',
 'https://www.ncbi.nlm.nih.gov/nuccore/26245415',
 'https://www.ncbi.nlm.nih.gov/nuccore/20918',
 'https://www.ncbi.nlm.nih.gov/nuccore/24563533',
 'https://www.ncbi.nlm.nih.gov/nuccore/39874648',
 'https://www.ncbi.nlm.nih.gov/nuccore/4839978',
 'https://www.ncbi.nlm.nih.gov/nuccore/42049400',
 'https://www.ncbi.nlm.nih.gov/nuccore/42048491',
 'https://www.ncbi.nlm.nih.gov/nuccore/36516274']

In [240]:
import requests
id_content = []
for url in url_get_id(genes):
    r = requests.get(url)
    id_content.append(r.content)
# id_content

In [279]:
import requests
from bs4 import BeautifulSoup

# Making a GET request
#r = requests.get('https://www.ncbi.nlm.nih.gov/nuccore/{}'.format(genes))
# Parsing the HTML
locus = ""
locus_list = []
for c in id_content:
	soup = BeautifulSoup(c, 'html.parser')

# Procurar um tag meta com um determinado atributo

	lines = soup.find_all('meta', {'name':"ncbi_uidlist"} )

	id = ""
	url = ""
	for line in lines:
		# print(line)
		# if 'name' in line.attrs:
		# 	print(line.attrs['name'])
		if 'content' in line.attrs:
			# print(line.attrs['content'])		
			id = line.attrs['content']

	if id:
		url ="https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id={}&db=nuccore&report=genbank&conwithfeat=on&hide-cdd=on&retmode=text&ncbi_phid=CE88F25338A215A1000000000483042A&withmarkup=on&tool=portal&log$=seqview&maxdownloadsize=1000000".format(id)

	r2 = requests.get(url)
	locus += r2.content.decode('utf-8')

	if "Error" not in r2.content.decode('utf-8'):
		locus_list.append(r2.content.decode('utf-8'))
print(locus)
	#utf-8 para converter de bytes para string
	#https://stackoverflow.com/questions/31019854/typeerror-cant-use-a-string-pattern-on-a-bytes-like-object-in-re-findall

LOCUS       CSU02075                1780 bp    rRNA    linear   INV 28-JUN-1994
DEFINITION  Chlorarachnion sp. 1 CCMP 1408 clones pF58 and pF59 18S rRNA small
            subunit.
ACCESSION   U02075
VERSION     U02075
KEYWORDS    .
SOURCE      Chlorarachnion sp. 1
  ORGANISM  Chlorarachnion sp. 1
            Eukaryotae; mitochondrial eukaryotes; Chlorarachniophyceae;
            Chlorarachnion.
REFERENCE   1  (bases 1 to 1780)
  AUTHORS   McFadden,G.I., Gilson,P.R., Hofmann,C.J., Adcock,G.J. and
            Maier,U.G.
  TITLE     Evidence that an amoeba acquired a chloroplast by retaining part of
            an engulfed eukaryotic alga
  JOURNAL   Proc. Natl. Acad. Sci. U.S.A. 91, 3690-3694 (1994)
  MEDLINE   94224807
REFERENCE   2  (bases 1 to 1780)
  AUTHORS   McFadden,G.I.
  TITLE     Direct Submission
  JOURNAL   Submitted (15-NOV-1993) Geoffrey I. McFadden, School of Botany,
            University of Melbourne, Parkville, VIC 3052, Australia
COMMENT     [WARNING] On Jan 20, 1999 t

LINK NCBI

In [242]:
import re
id_lista = []
# print(genes)
existe = re.findall(r'ACCESSION\s+[^\s]+', locus)
#\s+ um ou mais espaços
#[^\s]+ indica um conjunto de caracteres que se podem repetir uma ou mais vezes

if existe:
    for c in existe:
        m = re.match(r'ACCESSION\s+([^\s]+)', c )
        if m:
            id = m.group(1)
            id_lista.append("https://www.ncbi.nlm.nih.gov/nuccore/{}".format( id ) )
else:
    print( "Padrão não encontrado" )
id_lista

print

['https://www.ncbi.nlm.nih.gov/nuccore/U02075',
 'https://www.ncbi.nlm.nih.gov/nuccore/X07874',
 'https://www.ncbi.nlm.nih.gov/nuccore/AY158036',
 'https://www.ncbi.nlm.nih.gov/nuccore/BW173609',
 'https://www.ncbi.nlm.nih.gov/nuccore/CK292830',
 'https://www.ncbi.nlm.nih.gov/nuccore/AQ527824',
 'https://www.ncbi.nlm.nih.gov/nuccore/AADB01087788',
 'https://www.ncbi.nlm.nih.gov/nuccore/AADB01088697',
 'https://www.ncbi.nlm.nih.gov/nuccore/AACN010823022']

ACCESSION

In [243]:
acc_list = []

existe = re.findall(r'ACCESSION\s+.*?(?=VERSION)', locus, re.DOTALL)
#\s+ um ou mais espaços
#\.* encontra o caractere ponto final zero ou mais vezes
#?= something que faz match com VERSION mas não consome estes carateres
if existe:
    for accession in existe:
        m = re.match( r'ACCESSION\s+(.+)', accession, re.DOTALL )
        acc_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
#(.+) agrupar um ou mais caracteres dentro de um grupo
#re.sub(r'\s+', ' ' ...) substitui um ou mais espaços por apenas um espaço
acc_list

['U02075 ',
 'X07874 ',
 'AY158036 ',
 'BW173609 ',
 'CK292830 ',
 'AQ527824 ',
 'AADB01087788 AADB01000000 ',
 'AADB01088697 AADB01000000 ',
 'AACN010823022 AACN010000000 ']

VERSION

In [244]:
version_list = []

existe = re.findall(r'VERSION.*?(?=KEYWORDS|DBLINK)', locus, re.DOTALL)
#\s+ um ou mais espaços
#\.* encontra o caractere ponto final zero ou mais vezes
#?= something que faz match com VERSION mas não consome estes carateres
if existe:
    for vers in existe:
        m = re.match( r'VERSION\s+(.+)', vers, re.DOTALL )
        version_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
#(.+) agrupar um ou mais caracteres dentro de um grupo
#re.sub(r'\s+', ' ' ...) substitui um ou mais espaços por apenas um espaço
version_list

['U02075 ',
 'X07874.2 ',
 'AY158036.1 ',
 'BW173609.1 ',
 'CK292830.1 ',
 'AQ527824.1 ',
 'AADB01087788.1 ',
 'AADB01088697.1 ',
 'AACN010823022.1 ']

DEFINITION

In [245]:
defi_list = []
existe = re.findall(r'DEFINITION\s+.*?(?=ACCESSION)', locus, re.DOTALL)
if existe:
    for definition in existe:
        m = re.match( r'DEFINITION\s+(.+)', definition, re.DOTALL )
        defi_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
defi_list

['Chlorarachnion sp. 1 CCMP 1408 clones pF58 and pF59 18S rRNA small subunit. ',
 "Human CD2 gene exon 5 and 3'-flanking region. ",
 'Gamma proteobacterium UQ1 16S ribosomal RNA gene, partial sequence. ',
 "BW173609 Nori Satoh unpublished cDNA library, neural complex Ciona intestinalis cDNA clone rcinc027k05 3', mRNA sequence. ",
 "EST755544 Nicotiana benthamiana mixed tissue cDNA library, normalized, full-length Nicotiana benthamiana cDNA clone NBMCC27 5' end, mRNA sequence. ",
 'RPCI-11-348E6.TJ RPCI-11 Homo sapiens genomic clone RPCI-11-348E6, genomic survey sequence. ',
 'Homo sapiens chromosome 18 WGA:87787 contig 87787 scaffold 3690, whole genome shotgun sequence. ',
 'Homo sapiens chromosome 18 WGA:88696 contig 88696 scaffold 3714, whole genome shotgun sequence. ',
 'Canis lupus familiaris breed poodle ctg19866851636852, whole genome shotgun sequence. ']

LOCUS

In [246]:
loc_list = []
existe = re.findall(r'LOCUS\s+.*?(?=DEFINITION)', locus, re.DOTALL)
if existe:
    for loc in existe:
        m = re.match( r'LOCUS\s+(.+)', loc, re.DOTALL )
        loc_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
loc_list

['CSU02075 1780 bp rRNA linear INV 28-JUN-1994 ',
 'HSCD2G5 807 bp DNA linear PRI 30-SEP-1993 ',
 'AY158036 502 bp DNA linear BCT 08-DEC-2002 ',
 'BW173609 778 bp mRNA linear EST 07-JAN-2011 ',
 'CK292830 807 bp mRNA linear EST 02-AUG-2004 ',
 'AQ527824 510 bp DNA linear GSS 18-MAY-1999 ',
 'AADB01087788 27696 bp DNA linear PRI 03-FEB-2004 ',
 'AADB01088697 67145 bp DNA linear PRI 03-FEB-2004 ',
 'AACN010823022 816 bp DNA linear MAM 27-SEP-2003 ']

LOCUS NAME

In [280]:
loc_name_list = []
existe = re.findall(r'LOCUS\s + [^\s]+', locus, re.DOTALL)
if existe:
    for id in existe:
        m = re.match( r'LOCUS\s+(.+)', id, re.DOTALL )
        loc_name_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
loc_name_list

['CSU02075',
 'HSCD2G5',
 'AY158036',
 'BW173609',
 'CK292830',
 'AQ527824',
 'AADB01087788',
 'AADB01088697',
 'AACN010823022']

NUMBER OF BP

In [248]:
loc_bp_list = []
loc = []

existe = re.findall(r'LOCUS\s+.*?(?=DEFINITION)', locus, re.DOTALL)
# print(existe)
if existe:
    for c in existe:
        loc.append(list(filter(None, c.split(' '))))
for x in range(len(loc)):
    loc_bp_list.append(' '.join(loc[x][2:4])) 

#cria lista com os valores do locus, e escolhe os 3º e 4º caracteres, que correspondem ao nr de bp
loc_bp_list

['1780 bp',
 '807 bp',
 '502 bp',
 '778 bp',
 '807 bp',
 '510 bp',
 '27696 bp',
 '67145 bp',
 '816 bp']

MOLECULE TYPE

In [249]:
mol_type = []
for m in range(len(loc)):
    if "linear" in loc[m][5]:
        mol_type.append(' '.join(loc[m][4:6]))
    else:
        mol_type.append(' '.join(loc[m][4:5]))
#temos que corrigir esta porque as mol types podem ser de uma, duas ou trÊs palavras..
mol_type

['rRNA linear',
 'DNA linear',
 'DNA linear',
 'mRNA linear',
 'mRNA linear',
 'DNA linear',
 'DNA linear',
 'DNA linear',
 'DNA linear']

GENBANK DIVISION

In [250]:
gb_div = []
for gb in range(len(loc)):
    gb_div.append(' '.join(loc[gb][6:7]))
gb_div

['INV', 'PRI', 'BCT', 'EST', 'EST', 'GSS', 'PRI', 'PRI', 'MAM']

MODIFICATION DATE

In [251]:
mod_date = []
for md in range(len(loc)):
    mod_date.append(''.join(loc[md][7]).replace("\n",""))
mod_date

['28-JUN-1994',
 '30-SEP-1993',
 '08-DEC-2002',
 '07-JAN-2011',
 '02-AUG-2004',
 '18-MAY-1999',
 '03-FEB-2004',
 '03-FEB-2004',
 '27-SEP-2003']

SOURCE

In [252]:
source_list = []
existe = re.findall(r'SOURCE\s+.*?(?=ORGANISM)', locus, re.DOTALL)
# print(existe)
if existe:
    for source in existe:
        m = re.match( r'SOURCE\s+(.+)', source, re.DOTALL )
        source_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
source_list

['Chlorarachnion sp. 1 ',
 'Unknown. ',
 'gamma proteobacterium UQ1 ',
 'Ciona intestinalis (vase tunicate) ',
 'Nicotiana benthamiana ',
 'Homo sapiens (human) ',
 'Homo sapiens (human) ',
 'Homo sapiens (human) ',
 'Canis lupus familiaris (dog) ']

ORGANISM

In [253]:
org_list = []
existe = re.findall(r'ORGANISM\s+.*?(?=\n)', locus, re.DOTALL)
if existe:
    for org in existe:
        m = re.match( r'ORGANISM\s+(.+)', org, re.DOTALL )
        org_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
org_list

['Chlorarachnion sp. 1',
 'Unknown.',
 'gamma proteobacterium UQ1',
 'Ciona intestinalis',
 'Nicotiana benthamiana',
 'Homo sapiens',
 'Homo sapiens',
 'Homo sapiens',
 'Canis lupus familiaris']

FEATURES

In [254]:
misc = []
existe = re.findall(r'FEATURES\s+.*?(?=ORIGIN)', locus, re.DOTALL)
#DOTALL ignora as linhas, pois as features ocupam várias linhas
if existe:
    for feat in existe:
        m = re.match( r'FEATURES\s+(.+)',feat, re.DOTALL )
        misc.append(re.sub(r'/',  ' ', m.group(1) ) )
    for c in misc:
        print(c, "\n")

Location Qualifiers
     source          1..1780
                      organism="Chlorarachnion sp. 1"
                      mol_type="rRNA"
                      strain="CCMP 1408"
                      db_xref="taxon:29200"
                      clone="pF58 (Nu), pF59 (Nu)"
     rRNA            1..1780
                      product="18S rRNA small subunit"
 

Location Qualifiers
     source          1..807
                      organism="unknown"
                      mol_type="unassigned DNA"
     source          1..807
                      clone="CD2-cos1, CD2-cos4, CD2-cos7 and CD2-cos10"
                      organism="Homo sapiens"
     precursor_RNA   <1..>339
                      note="primary transcript"
     intron          <1..22
                      note="Intron IV (ca. 3.9 kb)"
     mRNA            23..>339
                      note="Exon 5"
     misc_feature    770..775
                      note="pot. polyA signal"
 

Location Qualifiers
     source          1..502


FEATURES - SOURCE

In [255]:

existe = re.findall(r'source\s+[1-9\.]*', locus, re.DOTALL)
if existe:
    for fsource in existe:
        print( re.sub(r'source\s+', ' ', fsource) )


 1..178
 1..8
 1..8
 1..5
 1..778
 1..8
 1..51
 1..27696
 1..67145
 1..816


FEATURES - EXONS

In [ ]:
existe = re.findall(r'exon\s+[1-9\>\.]*', locus, re.DOTALL)
if existe:
    for fexons in existe:
        print( re.sub(r'exon\s+', ' ', fexons) )

FEATURES - INTRON

In [ ]:
existe = re.findall(r'intron\s+[1-9\>\.]*', locus, re.DOTALL)
if existe:
    for fintrons in existe:
        print( re.sub(r'intron\s+', ' ', fintrons) )

CDS - JOIN

In [ ]:
existe = re.findall(r'CDS\s+?(join)[1-9\.\,\(\)]*', locus, re.DOTALL)
#(join) procura a palavra join 1x
#procura nrs de 1-9, ponto, virgula e parentises zero ou mais vezes
print(existe[0])
if existe:
    for cds in existe:
        print( re.sub(r'CDS\s+', ' ', cds) )

SEQUENCE/ORIGIN

In [256]:
seq_origin = []
existe = re.findall(r'ORIGIN\s+.*?(?=//)', locus, re.DOTALL)
if existe:
    for nuc in existe:
        m = re.match( r'ORIGIN\s+(.+)', nuc, re.DOTALL)
        a = re.sub(r'\s+', '', m.group(1) ) 
        num = re.sub(r'\d+','', a)
        seq_origin.append(num)
seq_origin

['...']

In [257]:
seq_count = []
for seq in seq_origin:
    #contagem de nucleotidos
    A= []
    C= []
    T= []
    G= []
    for n in seq:
        if n == 'a':
            A.append('a')
        if n == 'c':
            C.append('c')
        if n == 't':
            T.append('t')
        if n == 'g':
            G.append('g')
        
    a = len(A)
    c = len(C)
    t = len(T)
    g = len(G)
    seq_count.append([a,c,t,g])
    #fazer append de um array
seq_count


[[457, 366, 484, 472],
 [204, 228, 196, 179],
 [130, 110, 99, 163],
 [259, 130, 209, 179],
 [200, 145, 214, 248],
 [158, 93, 171, 84],
 [7805, 5568, 8619, 5704],
 [21574, 14059, 17953, 13559],
 [220, 180, 279, 137]]

In [258]:
length = []
for seq in seq_origin:
    length.append(len(seq))
length

[1780, 807, 502, 778, 807, 510, 27696, 67145, 816]

JOURNAL

In [259]:
journ_list = []
existe = re.findall(r'JOURNAL\s+.*?(?=PUBMED|REFERENCE|FEATURES|COMMENT)', locus, re.DOTALL)
if existe:
    for journal in existe:
        m = re.match( r'JOURNAL\s+(.+)', journal, re.DOTALL )
        journ_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
journ_list
#| dá-nos operador ou


['Proc. Natl. Acad. Sci. U.S.A. 91, 3690-3694 (1994) MEDLINE 94224807 ',
 'Submitted (15-NOV-1993) Geoffrey I. McFadden, School of Botany, University of Melbourne, Parkville, VIC 3052, Australia ',
 'EMBO J. 7 (6), 1675-1682 (1988) MEDLINE 89005055 ',
 'Unpublished ',
 'Submitted (02-OCT-2002) Molecular Biology, PROIMI, Av. Belgrano y Pje. Caseros, S.M. de Tucuman, Tucuman 4000, Argentina ',
 'Unpublished (2002) ',
 'Unpublished ',
 'Unpublished ',
 'Science 291 (5507), 1304-1351 (2001) ',
 'Submitted (25-NOV-2003) Celera, 45 W Gude Drive, Rockville, MD 20850, USA ',
 'Science 291 (5507), 1304-1351 (2001) ',
 'Submitted (25-NOV-2003) Celera, 45 W Gude Drive, Rockville, MD 20850, USA ',
 'Science 301 (5641), 1898-1903 (2003) ',
 'Submitted (09-MAY-2003) Department of Eukaryotic Genomics, TIGR, The Institute for Genomic Research, 9712 Medical Center Drive, Rockville, MD 20850, USA ']

AUTHORS

In [178]:
authors_list = []
existe = re.findall(r'AUTHORS\s+.*?(?=TITLE)', locus, re.DOTALL)
if existe:
    for auth in existe:
        m = re.match( r'AUTHORS\s+(.+)', auth, re.DOTALL )
        authors_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
authors_list

['McFadden,G.I., Gilson,P.R., Hofmann,C.J., Adcock,G.J. and Maier,U.G. ',
 'McFadden,G.I. ',
 'Lang,G., Wotton,D., Owen,M.J., Sewell,W.A., Brown,M.H., Mason,D.Y., Crumpton,M.J. and Kioussis,D. ',
 'Benito,J.M., Lovrich,G.A., Abate,C.M. and Sineriz,F. ',
 'Benito,J.M., Lovrich,G.A., Abate,C.M. and Sineriz,F. ',
 'Satou,Y., Shin-i,T., Kohara,Y. and Satoh,N. ',
 'Buell,C.R., Hart,A., Zismann,V., Karamycheva,S.A., Day,B., Staskawicz,B., Jin,H. and Baker,B. ',
 'Zhao,S., Adams,M.D., Nierman,W., Malek,J., de Jong,P. and Venter,J.C. ',
 'Venter,J.C., Adams,M.D., Myers,E.W., Li,P.W., Mural,R.J., Sutton,G.G., Smith,H.O., Yandell,M., Evans,C.A., Holt,R.A., Gocayne,J.D., Amanatides,P., Ballew,R.M., Huson,D.H., Wortman,J.R., Zhang,Q., Kodira,C.D., Zheng,X.H., Chen,L., Skupski,M., Subramanian,G., Thomas,P.D., Zhang,J., Gabor Miklos,G.L., Nelson,C., Broder,S., Clark,A.G., Nadeau,J., McKusick,V.A., Zinder,N., Levine,A.J., Roberts,R.J., Simon,M., Slayman,C., Hunkapiller,M., Bolanos,R., Delcher,A., Dew

TITLE

In [179]:
titles = []
existe = re.findall(r'TITLE\s+.*?(?=JOURNAL)', locus, re.DOTALL)
if existe:
    for title in existe:
        m = re.match( r'TITLE\s+(.+)', title, re.DOTALL )
        titles.append(re.sub(r'\s+', ' ', m.group(1) ) )
titles

['Evidence that an amoeba acquired a chloroplast by retaining part of an engulfed eukaryotic alga ',
 'Direct Submission ',
 'The structure of the human CD2 gene and its expression in transgenic mice ',
 'Microbial populations from the subantarctic marine ecosystem ',
 'Direct Submission ',
 'Expressed genes in Ciona intestinalis (2002c) ',
 'Generation of EST sequences from Nicotiana benthamiana ',
 'Use of BAC End Sequences from Library RPCI-11 for Sequence-Ready Map Building ',
 'The sequence of the human genome ',
 'Direct Submission ',
 'The sequence of the human genome ',
 'Direct Submission ']

PUBMED

In [284]:
pub_list = []
existe = re.findall(r'PUBMED\s+.*?(?=REFERENCE)', locus, re.DOTALL)
# print(existe)
if existe:
    for pubm in existe:
        m = re.match( r'PUBMED\s+(.+)', pubm, re.DOTALL )
        pub_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
print(pub_list)

#procurar os ficheiros que tem o campo PUBMED, se não tiver vai valor vazio na lista (para fazer a relação genbank-PM)
file_locus_pub_list = []
for x in locus_list:
    existe = re.findall(r'PUBMED\s+.*?(?=REFERENCE)', x, re.DOTALL)
    # print(existe)
    if existe:
        for pubm in existe:
            m = re.match( r'PUBMED\s+(.+)', pubm, re.DOTALL )
            file_locus_pub_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
    else:
        file_locus_pub_list.append('')
print(file_locus_pub_list)

['11181995 ', '11181995 ', '14512627 ']
['', '', '', '', '', '', '11181995 ', '11181995 ', '14512627 ']


In [261]:
import requests
from bs4 import BeautifulSoup

def url_get_id_pm(item):
    url_id_pm = []
    for id_pm in item:
        url = f"https://pubmed.ncbi.nlm.nih.gov/{id_pm}/"
        url_id_pm.append(url)
    return url_id_pm

pm = ""
soup = []
content_pm = []
urls = url_get_id_pm(pub_list)
print(urls)

for url in urls:
	r2 = requests.get(url)
	pm += r2.content.decode('utf-8')
	soup.append(BeautifulSoup(r2.content, 'html.parser'))
	content_pm.append(r2.content.decode('utf-8'))

#print(soup)
#print(content_pm)




['https://pubmed.ncbi.nlm.nih.gov/11181995 /', 'https://pubmed.ncbi.nlm.nih.gov/11181995 /', 'https://pubmed.ncbi.nlm.nih.gov/14512627 /']


AFFILIATION

In [151]:
import requests
r = requests.get('https://pubmed.ncbi.nlm.nih.gov/2549509')
pub = r.content.decode('utf-8')
print(pub)

<!DOCTYPE html>




<html lang="en" >
<head  itemscope itemtype="http://schema.org/WebPage"  prefix="og: http://ogp.me/ns#" >
    <meta charset="UTF-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">

    <!-- Mobile properties -->
    <meta name="HandheldFriendly" content="True">
    <meta name="MobileOptimized" content="320">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">

  
  
  <link rel="preconnect" href="https://cdn.ncbi.nlm.nih.gov">
  <link rel="preconnect" href="https://www.ncbi.nlm.nih.gov">
  <link rel="preconnect" href="https://www.google-analytics.com">

  
  
    <link rel="stylesheet" href="https://cdn.ncbi.nlm.nih.gov/pubmed/8ae01552-7288-4f29-8763-ac14f06ac5ea/CACHE/css/output.e3e95ed4a5de.css" type="text/css">
  

  <link rel="stylesheet" href="https://cdn.ncbi.nlm.nih.gov/pubmed/8ae01552-7288-4f29-8763-ac14f06ac5ea/CACHE/css/output.452c70ce66f7.css" type="text/css">

  
    
  

  
    <link rel="stylesheet" href="https://cdn.n

In [183]:
import re
aff_list = []
for x in content_pm:
    existe = re.findall(r'"affiliation-\d"><sup class="key">\d</sup> \s*.*?(?=[.])', x, re.DOTALL)
#\s* zero ou mais espaços
#.* zero ou mais caracteres
#\. caractere ponto final -> procura affiliation até encontrar ponto final
    # print(existe)
    if existe:
        aux_list = []
        for c in existe:
        #m = re.findall(r'>\s*', c[0] )
        #o findall retorna uma lista, por isso temos que selecionar m[0] para fazer o replace
            m = c.replace('affiliation-1"><sup class="key">1</sup>','')
            aux_list.append(m)   #preenche para cada id a affiliation (mesmo que repetida)

    aux_list = list(dict.fromkeys(aux_list))  #remove os duplicados anteriormente inseridos
    aff_list.append(aux_list)

print(aff_list)
    # else:
        # print( "Padrão não encontrado" )


[['" Celera Genomics, 45 West Gude Drive, Rockville, MD 20850, USA'], ['" Celera Genomics, 45 West Gude Drive, Rockville, MD 20850, USA']]


AUTHOR NAME

In [188]:
auth_list = []
for a in content_pm:
    #print(a)
    existe = re.findall(r'"full-name"(.*?)>[^\n][\w\s\.\;]*?</a>', a, re.DOTALL)
    # print(existe)
    aux_auth_list = []
    if existe:
        for c in existe:
            m = re.findall(r'>[^\n][\w\s\.\;]+</', c)
            # print(m)
            for e in m:
                e = e.replace('>','')
                e = e.replace('</','')
                aux_auth_list.append(e)
    auth_list.append(aux_auth_list)
print(aux_auth_list)

['J C Venter', '&nbsp;et al.', '2001', 'Show details', 'Display options', 'Abstract', 'PubMed', 'PMID', 'Actions', '. ']


In [262]:
import re
from Bio import SeqIO
from Bio import Entrez
from Bio import Medline
import pandas as pd
import numpy as np

In [263]:
Ids=[]
title_list = []
abstract = []
author = []
affiliation = []
journal = []
database = 'PubMed'
Entrez.email= "pg49836@alunos.uminho.pt"
# idlist= pub_list
idlist = ["36607602","36603771"]
handle = Entrez.efetch(db=database, id=idlist, rettype="medline", retmode="text") 
records = Medline.parse(handle)
description=[]
for info in records:
    title_list.append(info.get("TI", "-"))
    author.append(info.get("AU", "-"))
    journal.append(info.get("SO", "-"))
    affiliation.append(info.get("AD", "-") )
    abstract.append(info.get("AB", "-"))
    print()
print(title_list)
print(abstract)
print(author)
print(affiliation)
print(journal)




['AKT1-FOXO4 axis reciproaclly regulates hemochorial placentation.', 'A small molecule that targets the processivity factor of molluscum contagiosum virus has therapeutic potential.']
['Hemochorial placentation involves the differentiation of specialized cells called invasive trophoblast cells possessing the capacity to exit the placenta and invade into the uterus where they restructure the vasculature. Invasive trophoblast cells arise from a well-defined compartment within the placenta, referred to as the junctional zone in the rat and the extravillous trophoblast cell column in the human. In this study, we investigated roles for AKT1, a serine/threonine kinase, in placental development using a genome-edited/loss-of-function rat model. Disruption of AKT1 resulted in placental, fetal, and postnatal growth restriction. Forkhead box O4 (Foxo4), which encodes a transcription factor and known AKT substrate, was abundantly expressed in the junctional zone and invasive trophoblast cells of

CONNECTION TO DATABASE

In [264]:
import mysql.connector as SQLC
try: 
    DataBase = SQLC.connect(
    host ="127.0.0.1",
    user ="user",
    password ="password",
        database ="teste"
    )
    Cursor = DataBase.cursor()

    TableName ="""CREATE TABLE IF NOT EXISTS `LOCUS` (
        `Id_locus` INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
        `Accession` VARCHAR(100) NOT NULL,
        `Sequence_length` INT NULL,
        `Molecule_type` VARCHAR(20) NULL,
        `GB_division` VARCHAR(10) NULL,
        `Source` VARCHAR(45) NULL,
        `Organism` VARCHAR(45) NULL,
        `Locus_name` VARCHAR(45) NULL
        );
    """
    Cursor.execute(TableName)
    sql = "INSERT INTO LOCUS (Locus_name, Sequence_length, Molecule_type, GB_division, Source, Organism, Accession) VALUES"
    for index in range(len(loc_name_list)):
        if index == len(loc_name_list)-1:
            sql += f" (\"{loc_name_list[index]}\", \"{length[index]}\", \"{mol_type[index]}\", \"{gb_div[index]}\", \"{source_list[index]}\", \"{org_list[index]}\", \"{acc_list[index]}\")"
        else:
            sql += f" (\"{loc_name_list[index]}\", \"{length[index]}\", \"{mol_type[index]}\", \"{gb_div[index]}\", \"{source_list[index]}\", \"{org_list[index]}\", \"{acc_list[index]}\"),"
    print(sql)
    Cursor.execute(sql)

    DataBase.commit()

    print(Cursor.rowcount, "record inserted.")

    Cursor.close()

except SQLC.Error as error:
    print("Failed to insert record into MySQL table {}".format(error))

INSERT INTO LOCUS (Locus_name, Sequence_length, Molecule_type, GB_division, Source, Organism, Accession) VALUES ("CSU02075", "1780", "rRNA linear", "INV", "Chlorarachnion sp. 1 ", "Chlorarachnion sp. 1", "U02075 "), ("HSCD2G5", "807", "DNA linear", "PRI", "Unknown. ", "Unknown.", "X07874 "), ("AY158036", "502", "DNA linear", "BCT", "gamma proteobacterium UQ1 ", "gamma proteobacterium UQ1", "AY158036 "), ("BW173609", "778", "mRNA linear", "EST", "Ciona intestinalis (vase tunicate) ", "Ciona intestinalis", "BW173609 "), ("CK292830", "807", "mRNA linear", "EST", "Nicotiana benthamiana ", "Nicotiana benthamiana", "CK292830 "), ("AQ527824", "510", "DNA linear", "GSS", "Homo sapiens (human) ", "Homo sapiens", "AQ527824 "), ("AADB01087788", "27696", "DNA linear", "PRI", "Homo sapiens (human) ", "Homo sapiens", "AADB01087788 AADB01000000 "), ("AADB01088697", "67145", "DNA linear", "PRI", "Homo sapiens (human) ", "Homo sapiens", "AADB01088697 AADB01000000 "), ("AACN010823022", "816", "DNA linea

In [265]:
import mysql.connector as SQLC
try: 
    DataBase = SQLC.connect(
    host ="127.0.0.1",
    user ="user",
    password ="password",
        database ="teste"
    )
    Cursor = DataBase.cursor()

    TableName ="""CREATE TABLE IF NOT EXISTS `GENBANK` (
        `Id_GB` INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
        `GB_id` VARCHAR(45) NOT NULL,
        `Definition` VARCHAR(400) NULL,
        `Modification_date` VARCHAR(11) NULL,
        `Version` VARCHAR(20) NOT NULL UNIQUE,
        `LOCUS_Id_locus` INT NOT NULL,
        `ORIGIN_Id_origin` INT NULL
    )
    """
    Cursor.execute(TableName)
    sql = "INSERT INTO GENBANK (GB_Id, Modification_date, Definition, Version, LOCUS_Id_locus, ORIGIN_Id_origin) VALUES"
    

    for index in range(len(version_list)):
        sql_insert_origin = "INSERT INTO ORIGIN (Sequence, Length, Adenine, Citosine, Timine , Guanine) VALUES" 
        sql_insert_origin += f" (\"{seq_origin[index]}\", \"{length[index]}\", \"{seq_count[index][0]}\", \"{seq_count[index][1]}\", \"{seq_count[index][2]}\",  \"{seq_count[index][3]}\")"
        Cursor.execute(sql_insert_origin)
        # print(Cursor.lastrowid)  #buscar o id do origin inserido
        id_inserted_origin = Cursor.lastrowid


        select_locus = f"SELECT Id_locus from LOCUS where Locus_name = '{loc_name_list[index]}'"
        Cursor.execute(select_locus)
        myresult = Cursor.fetchall()
        for res, in myresult:
            id_locus = res
        # print(id_locus)
        
        if index == len(version_list)-1:
            sql += f" (\"{loc_name_list[index]}\", \"{mod_date[index]}\", \"{defi_list[index]}\", \"{version_list[index]}\", \"{id_locus}\", \"{id_inserted_origin}\")"
        else:
            sql += f" (\"{loc_name_list[index]}\", \"{mod_date[index]}\", \"{defi_list[index]}\", \"{version_list[index]}\", \"{id_locus}\", \"{id_inserted_origin}\"),"
    # print(sql)
    Cursor.execute(sql)

    DataBase.commit()

    print(Cursor.rowcount, "record inserted.")

    Cursor.close()

except SQLC.Error as error:
    print("Failed to insert record into MySQL table {}".format(error))

9 record inserted.


In [266]:
import mysql.connector as SQLC
try: 
    DataBase = SQLC.connect(
    host ="127.0.0.1",
    user ="user",
    password ="password",
        database ="teste"
    )
    Cursor = DataBase.cursor()

    TableName ="""CREATE TABLE IF NOT EXISTS `PUBMED` (
        `Id_PM` INT NOT NULL AUTO_INCREMENT,
        `PM_ID` INT NOT NULL,
        `Journal` VARCHAR(100) NULL,
        `Title` VARCHAR(500) NULL,
        `Abstract` LONGTEXT NULL,
        PRIMARY KEY (`Id_PM`)
    )
    """
    Cursor.execute(TableName)
    sql = "INSERT INTO PUBMED (PM_ID, Journal, Title, Abstract) VALUES"
    
    print(journal)
    print(len(title_list))
    print(len(abstract))
    print(len(pub_list))
    pub_list = list(dict.fromkeys(pub_list))  #tirar os duplicados
    for index in range(len(pub_list)):
        if index == len(pub_list)-1:
            sql += f" (\"{pub_list[index]}\", \"{journal[index]}\", \"{title_list[index]}\", \"{abstract[index]}\")"
        else:
            sql += f" (\"{pub_list[index]}\", \"{journal[index]}\", \"{title_list[index]}\", \"{abstract[index]}\"),"
    # print(sql)
    Cursor.execute(sql)

    DataBase.commit()

    print(Cursor.rowcount, "record inserted.")

    Cursor.close()

except SQLC.Error as error:
    print("Failed to insert record into MySQL table {}".format(error))

['Development. 2023 Jan 6:dev.201095. doi: 10.1242/dev.201095.', 'Antiviral Res. 2023 Jan 2:105520. doi: 10.1016/j.antiviral.2022.105520.']
2
2
3
2 record inserted.


In [287]:
import mysql.connector as SQLC
try: 
    DataBase = SQLC.connect(
    host ="127.0.0.1",
    user ="user",
    password ="password",
        database ="teste"
    )
    Cursor = DataBase.cursor()

    TableName ="""CREATE TABLE IF NOT EXISTS `AUTHORS` (
        `Id_Author` INT NOT NULL AUTO_INCREMENT,
        `Name` VARCHAR(45) NULL,
        `AFFILIATION_Id_affiliation` INT NULL,
        PRIMARY KEY (`Id_Author`)
    )
    """
    Cursor.execute(TableName)
    print(len(file_locus_pub_list))
    print(len(loc_name_list))

    for index_ids in range(len(file_locus_pub_list)):
        if file_locus_pub_list[index_ids] != '':
            select_pm_gb = f"SELECT Id_PM FROM PUBMED WHERE PM_ID = \"{file_locus_pub_list[index_ids]}\""
            Cursor.execute(select_pm_gb)
            myresult_pm = Cursor.fetchall()
            id_pm = None
            for res, in myresult_pm:
                id_pm = res

            select_gb_pm = f"SELECT Id_GB FROM GENBANK WHERE GB_ID= \"{loc_name_list[index_ids]}\""
            Cursor.execute(select_gb_pm)
            myresult_gb = Cursor.fetchall()
            id_gb = None
            for res, in myresult_gb:
                id_gb = res

            if id_gb is not None and id_pm is not None:      
                sql_pm_gb_relation = "INSERT INTO pm_gb_relationship (PUBMED_Id_PM, GENBANK_Id_GB) VALUES" 
                sql_pm_gb_relation += f" ({id_pm}, {id_gb})"
                print(sql_pm_gb_relation)
                Cursor.execute(sql_pm_gb_relation)

    
    DataBase.commit()

    print(Cursor.rowcount, "record inserted.")

    Cursor.close()

except SQLC.Error as error:
    print("Failed to insert record into MySQL table {}".format(error))

9
9
INSERT INTO pm_gb_relationship (PUBMED_Id_PM, GENBANK_Id_GB) VALUES (2, 23)
Failed to insert record into MySQL table 1062 (23000): Duplicate entry '2-23' for key 'pm_gb_relationship.PRIMARY'


In [288]:
import mysql.connector as SQLC
try: 
    DataBase = SQLC.connect(
    host ="127.0.0.1",
    user ="user",
    password ="password",
        database ="teste"
    )
    Cursor = DataBase.cursor()

    TableName ="""CREATE TABLE IF NOT EXISTS `AUTHORS` (
        `Id_Author` INT NOT NULL AUTO_INCREMENT,
        `Name` VARCHAR(45) NULL,
        `AFFILIATION_Id_affiliation` INT NULL,
        PRIMARY KEY (`Id_Author`)
    )
    """
    Cursor.execute(TableName)

    for index_pubmed in range(len(pub_list)):
        
        affiliation_ids = []
        for index in range(len(affiliation[index_pubmed])):

            #verificar se existe afiliação (evitar duplicados)
            select_affiliation = f"SELECT Id_affiliation FROM AFFILIATION WHERE Description = \"{affiliation[index_pubmed][index]}\""
            Cursor.execute(select_affiliation)
            myresult_af = Cursor.fetchall()
            id_affiliation = None
            for res, in myresult_af:
                id_affiliation = res


            if id_affiliation is not None:
                affiliation_ids.append(id_affiliation)
            else:
                sql_insert_affiliation = "INSERT INTO AFFILIATION (Description) VALUES" 
                sql_insert_affiliation += f" (\"{affiliation[index_pubmed][index]}\")"
                print(sql_insert_affiliation)
                Cursor.execute(sql_insert_affiliation)
                # print(Cursor.lastrowid)  #buscar o id do affiliation inserido
                affiliation_ids.append(Cursor.lastrowid)

        sql = "INSERT INTO AUTHORS (Name, AFFILIATION_Id_affiliation ) VALUES"
        for index in range(len(author[index_pubmed])):
            if index > len(affiliation_ids)-1:
                if index == len(author[index_pubmed])-1:
                    sql += f" (\"{author[index_pubmed][index]}\", null)"
                else:
                    sql += f" (\"{author[index_pubmed][index]}\", null),"
            else:
                if index == len(author[index_pubmed])-1:
                    sql += f" (\"{author[index_pubmed][index]}\", {affiliation_ids[index]})"
                else:
                    sql += f" (\"{author[index_pubmed][index]}\", {affiliation_ids[index]}),"
        print(sql)
        Cursor.execute(sql)

    DataBase.commit()

    print(Cursor.rowcount, "record inserted.")

    Cursor.close()

except SQLC.Error as error:
    print("Failed to insert record into MySQL table {}".format(error))

INSERT INTO AUTHORS (Name, AFFILIATION_Id_affiliation ) VALUES ("Kozai K", 13), ("Moreno-Irusta A", 13), ("Iqbal K", 13), ("Winchester ML", 14), ("Scott RL", 13), ("Simon ME", 13), ("Muto M", 13), ("Parrish MR", 14), ("Soares MJ", 13)
INSERT INTO AFFILIATION (Description) VALUES ("Department of Basic and Translational Sciences, School of Dental Medicine, University of Pennsylvania, USA.")
INSERT INTO AFFILIATION (Description) VALUES ("Perelman School of Medicine, University of Pennsylvania, USA.")
INSERT INTO AFFILIATION (Description) VALUES ("Fox Chase Chemical Diversity Center, USA.")
INSERT INTO AFFILIATION (Description) VALUES ("Conifer Point Pharmaceuticals, Doylestown, PA, USA.")
INSERT INTO AFFILIATION (Description) VALUES ("TR Bailey Consulting LLC, Phoenixville, PA, USA.")
INSERT INTO AFFILIATION (Description) VALUES ("Department of Basic and Translational Sciences, School of Dental Medicine, University of Pennsylvania, USA; Abramson Cancer Center, School of Medicine, Universi

IndexError: list index out of range